In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

from PIL import Image

### check interpreter

In [2]:
import sys
print(sys.executable)

D:\Projects\machine-learning-projects\face-mask-detection\venv\Scripts\python.exe


### This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset. Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.
### And then two Dense layers for classification.

In [3]:
model = Sequential([
    Conv2D(100, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(100, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### Image Data Generation/Augmentation:

In [4]:
TRAINING_DIR = "dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0 / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    target_size=(150, 150))

VALIDATION_DIR = "dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=10,
                                                              target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


### Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [5]:
checkpoint = ModelCheckpoint('model-{epoch:02d}.hdf5', monitor='val_loss', verbose=1, save_best_only=True,
                             mode='auto')

### Train the model:

In [6]:
history = model.fit_generator(train_generator,
                              epochs=2,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
132/132 [==============================] - ETA: 0s - loss: 0.6128 - acc: 0.6669
Epoch 00001: val_loss improved from inf to 0.22685, saving model to model-01.hdf5
132/132 [==============================] - 244s 2s/step - loss: 0.6128 - acc: 0.6669 - val_loss: 0.2268 - val_acc: 0.9485
Epoch 2/2
132/132 [==============================] - ETA: 0s - loss: 0.3076 - acc: 0.8852
Epoch 00002: val_loss improved from 0.22685 to 0.16133, saving model to model-02.hdf5
132/132 [==============================] - 403s 3s/step - loss: 0.3076 - acc: 0.8852 - val_loss: 0.1613 - val_acc: 0.9742
